<a href="https://colab.research.google.com/github/SoumyaAIDev/Hugging_Face_LLM_models/blob/main/sey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("braindao/solidity-dataset")
print(dataset['train'].to_pandas().head(1000))

README.md:   0%|          | 0.00/825 [00:00<?, ?B/s]

SolidityP.parquet:   0%|          | 0.00/620M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/99485 [00:00<?, ? examples/s]

                                                expert  \
0    Create a smart contract that extends the WETHO...   
1    Implement a smart contract that inherits from ...   
2    Implement a smart contract that inherits from ...   
3    Create a Solidity smart contract that allows f...   
4    Design a smart contract that implements a crow...   
..                                                 ...   
995  Implement a smart contract that inherits from ...   
996  Design a Solidity smart contract that enables ...   
997  Implement a Solidity smart contract that perfo...   
998  Create a Solidity smart contract that provides...   
999  Implement a multisig wallet smart contract tha...   

                                               average  \
0    Create a smart contract that builds upon the W...   
1    Create a smart contract for a BlindAuctionMark...   
2    Create a smart contract that allows for basic ...   
3    Create a smart contract for managing decentral...   
4    Create a

In [ ]:
from datasets import load_dataset

# Replace 'dataset_name' with the actual dataset name, such as 'braindao/solidity-dataset'
dataset = load_dataset("braindao/solidity-dataset")

# View the dataset information
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['expert', 'average', 'beginner', 'content'],
        num_rows: 99485
    })
})


In [ ]:
from datasets import load_dataset

# Load the dataset (replace 'braindao/solidity-dataset' with your actual dataset name)
dataset = load_dataset("braindao/solidity-dataset")

# Extract the 'content' column from the 'train' split
content_data = dataset["train"]["content"]

# Display a few rows of the 'content' column
for idx, content in enumerate(content_data[:5], start=1):
    print(f"Smart Contract {idx}:\n{content}\n")

# Save the entire 'content' column to a text file
with open("content_smart_contracts.txt", "w") as file:
    for idx, content in enumerate(content_data, start=1):
        file.write(f"Smart Contract {idx}:\n{content}\n\n")

print("All smart contract content saved to 'content_smart_contracts.txt'.")


Smart Contract 1:
// contracts/bridge/L1Helper.sol
// SPDX-License-Identifier: MIT
// https://tornado.cash
pragma solidity ^0.7.0;
pragma abicoder v2;
import "omnibridge/contracts/helpers/WETHOmnibridgeRouter.sol";
/// @dev Extension for original WETHOmnibridgeRouter that stores TornadoPool account registrations.
contract L1Helper is WETHOmnibridgeRouter {
  event PublicKey(address indexed owner, bytes key);
  struct Account {
    address owner;
    bytes publicKey;
  }
  constructor(IOmnibridge _bridge,
    IWETH _weth,
    address _owner) WETHOmnibridgeRouter(_bridge, _weth, _owner) {}
  function register(Account memory _account) public {
    require(_account.owner == msg.sender, "only owner can be registered");
    _register(_account);
  }
  function wrapAndRelayTokens(address _receiver,
    bytes memory _data,
    Account memory _account) public payable {
    WETH.deposit{ value: msg.value }();
    bridge.relayTokensAndCall(address(WETH), _receiver, msg.value, _data);
    if (_acco

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


data = {
    "expert": [
        "Create a smart contract that extends the WETHOptimized contract and adds a transfer function.",
        "Implement a smart contract that inherits from ERC721 and adds token metadata functionality."
    ],
    "content": [
        "// contracts/bridge/L1Helper.sol\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract L1Helper { ... }",
        "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract ERC721Metadata is ERC721 { ... }"
    ]
}
dataset = Dataset.from_dict(data)


def preprocess_function(examples):
    inputs = [f"Instruction: {expert}  Code: {content}" for expert, content in zip(examples["expert"], examples["content"])]


    targets = examples["content"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

print(tokenized_dataset)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['expert', 'content', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})


In [ ]:
from transformers import Trainer, TrainingArguments
tokenized_dataset = dataset.map(preprocess_function, batched=True)
train_dataset = tokenized_dataset.select(range(2))
validation_dataset = tokenized_dataset.select(range(1, 2))

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

trainer.train()

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,27.938593
2,No log,27.082073
3,No log,26.641869


TrainOutput(global_step=3, training_loss=22.59881337483724, metrics={'train_runtime': 96.5579, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.031, 'total_flos': 812050808832.0, 'train_loss': 22.59881337483724, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()

print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 26.641868591308594, 'eval_runtime': 3.5044, 'eval_samples_per_second': 0.285, 'eval_steps_per_second': 0.285, 'epoch': 3.0}


In [ ]:
for expert_instruction in data["expert"]:
    input_ids = tokenizer(expert_instruction, return_tensors="pt").input_ids

    output_ids = model.generate(input_ids, max_new_tokens=512, temperature=0.7)

    predicted_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"Instruction: {expert_instruction}")
    print(f"Generated Solidity Code: {predicted_code}")
    print("-" * 50)

Instruction: Create a smart contract that extends the WETHOptimized contract and adds a transfer function.
Generated Solidity Code: a smart contract that extends the WETHOptimized contract and adds a transfer function.
--------------------------------------------------
Instruction: Implement a smart contract that inherits from ERC721 and adds token metadata functionality.
Generated Solidity Code: and adds token metadata functionality. Implement a smart contract that inherits from ERC721 and adds token metadata functionality.
--------------------------------------------------


In [ ]:
from datasets import Dataset


data = {
    "expert": [
        "Create a smart contract that extends the WETHOptimized contract and adds a transfer function.",
        "Implement a smart contract that inherits from ERC721 and adds token metadata functionality.",
        "Implement a smart contract that inherits from ERC721 and adds token metadata functionality.",
        "Create a Solidity smart contract that allows for token staking.",
        "Design a smart contract that implements a crowdfunding platform.",
        "Create a smart contract that implements the BKC token.",
        "Implement a smart contract that simulates a Coin Flip game.",

    ],
    "content": [
        "// contracts/bridge/L1Helper.sol\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract L1Helper { ... }",
        "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract ERC721Metadata is ERC721 { ... }",
        "// SPDX-License-Identifier: UNLICENSED\npragma solidity ^0.8.0; contract ERC721Advanced { ... }",
        "pragma solidity 0.4.24;\nimport '../kernel/IKernel.sol'; contract TokenStaking { ... }",
        "pragma solidity ^0.4.18;\nlibrary SafeMath {...} contract Crowdfunding { ... }",
        "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract BKCToken { ... }",
        "// SPDX-License-Identifier: AGPL-3.0-only\npragma solidity ^0.8.0; contract CoinFlip { ... }",

    ]
}


dataset = Dataset.from_dict(data)

content_list = [record["content"] for record in dataset]

for i, content in enumerate(content_list):
    print(f"Smart Contract {i + 1}:\n{content}\n")


Smart Contract 1:
// contracts/bridge/L1Helper.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0; contract L1Helper { ... }

Smart Contract 2:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0; contract ERC721Metadata is ERC721 { ... }

Smart Contract 3:
// SPDX-License-Identifier: UNLICENSED
pragma solidity ^0.8.0; contract ERC721Advanced { ... }

Smart Contract 4:
pragma solidity 0.4.24;
import '../kernel/IKernel.sol'; contract TokenStaking { ... }

Smart Contract 5:
pragma solidity ^0.4.18;
library SafeMath {...} contract Crowdfunding { ... }

Smart Contract 6:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0; contract BKCToken { ... }

Smart Contract 7:
// SPDX-License-Identifier: AGPL-3.0-only
pragma solidity ^0.8.0; contract CoinFlip { ... }



In [ ]:
from datasets import load_dataset

# Load the braindao/solidity-dataset
dataset = load_dataset("braindao/Solidity-Dataset")

# Access the training split
train_dataset = dataset["train"]

# Extract the 'content' field containing Solidity code
solidity_code_list = train_dataset["content"]

# Format and display all contracts
output = ""
for i, code in enumerate(solidity_code_list, start=1):
    formatted_contract = f"Smart Contract {i}:\n{code}\n\n"
    output += formatted_contract  # Append to the full output string
    print(formatted_contract)    # Print each contract

# Optional: Save the formatted dataset to a file
with open("formatted_smart_contracts.txt", "w") as file:
    file.write(output)

print("Formatted smart contracts saved to 'formatted_smart_contracts.txt'.")


README.md:   0%|          | 0.00/825 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
        address receiver,
        SwapQuote calldata quote) internal returns (uint256 amt) {
        if (address(quote.sellToken) == Adapter(adapter).target()) {
            amt = _amt;
        } else if (address(quote.sellToken) == Adapter(adapter).underlying()) {
            amt = Adapter(adapter).wrapUnderlying(_amt);
        } else {
            // sell tokens for underlying and wrap into target
            amt = Adapter(adapter).wrapUnderlying(_fillQuote(quote));
            // refund any remaining quote.sellToken to receiver
            _transfer(quote.sellToken,
                receiver,
                address(quote.sellToken) == ETH
                    ? address(this).balance
                    : _balanceOf(address(quote.sellToken), address(this)));
        }
    }
    /// @notice Given an amount and a quote, decides whether it needs to unwrap and make a swap on 0x,
    /// simply unwrap tokens or do nothing
    /// @dev when

In [ ]:
from datasets import load_dataset

# Load the braindao/solidity-dataset from Hugging Face
dataset = load_dataset("braindao/Solidity-Dataset")

# Extract the 'content' field from the dataset
content_list = [record["content"] for record in dataset["train"]]

# Print the first few Solidity contracts
for i, content in enumerate(content_list[:5]):  # Adjust the range as needed
    print(f"Smart Contract {i + 1}:\n{content}\n")


README.md:   0%|          | 0.00/825 [00:00<?, ?B/s]

Smart Contract 1:
// contracts/bridge/L1Helper.sol
// SPDX-License-Identifier: MIT
// https://tornado.cash
pragma solidity ^0.7.0;
pragma abicoder v2;
import "omnibridge/contracts/helpers/WETHOmnibridgeRouter.sol";
/// @dev Extension for original WETHOmnibridgeRouter that stores TornadoPool account registrations.
contract L1Helper is WETHOmnibridgeRouter {
  event PublicKey(address indexed owner, bytes key);
  struct Account {
    address owner;
    bytes publicKey;
  }
  constructor(IOmnibridge _bridge,
    IWETH _weth,
    address _owner) WETHOmnibridgeRouter(_bridge, _weth, _owner) {}
  function register(Account memory _account) public {
    require(_account.owner == msg.sender, "only owner can be registered");
    _register(_account);
  }
  function wrapAndRelayTokens(address _receiver,
    bytes memory _data,
    Account memory _account) public payable {
    WETH.deposit{ value: msg.value }();
    bridge.relayTokensAndCall(address(WETH), _receiver, msg.value, _data);
    if (_acco

In [ ]:
from datasets import load_dataset
dataset = load_dataset("braindao/solidity-dataset")
print(dataset['train'].to_pandas().head(1000))

                                                expert  \
0    Create a smart contract that extends the WETHO...   
1    Implement a smart contract that inherits from ...   
2    Implement a smart contract that inherits from ...   
3    Create a Solidity smart contract that allows f...   
4    Design a smart contract that implements a crow...   
..                                                 ...   
995  Implement a smart contract that inherits from ...   
996  Design a Solidity smart contract that enables ...   
997  Implement a Solidity smart contract that perfo...   
998  Create a Solidity smart contract that provides...   
999  Implement a multisig wallet smart contract tha...   

                                               average  \
0    Create a smart contract that builds upon the W...   
1    Create a smart contract for a BlindAuctionMark...   
2    Create a smart contract that allows for basic ...   
3    Create a smart contract for managing decentral...   
4    Create a

In [ ]:
from datasets import load_dataset
import os

# Load the braindao/solidity-dataset
dataset = load_dataset("braindao/Solidity-Dataset")

# Extract the 'content' field containing Solidity code
content_list = [record["content"] for record in dataset["train"]]

# Print the first few Solidity contracts
for i, content in enumerate(content_list[:5]):  # Adjust the range as needed
    print(f"Smart Contract {i + 1}:\n{content}\n")

# Save all contracts to individual files
output_dir = "solidity_contracts"
os.makedirs(output_dir, exist_ok=True)

for i, content in enumerate(content_list):
    file_path = os.path.join(output_dir, f"contract_{i + 1}.sol")
    with open(file_path, "w") as file:
        file.write(content)

print(f"Extracted {len(content_list)} Solidity contracts and saved to '{output_dir}'.")


Smart Contract 1:
// contracts/bridge/L1Helper.sol
// SPDX-License-Identifier: MIT
// https://tornado.cash
pragma solidity ^0.7.0;
pragma abicoder v2;
import "omnibridge/contracts/helpers/WETHOmnibridgeRouter.sol";
/// @dev Extension for original WETHOmnibridgeRouter that stores TornadoPool account registrations.
contract L1Helper is WETHOmnibridgeRouter {
  event PublicKey(address indexed owner, bytes key);
  struct Account {
    address owner;
    bytes publicKey;
  }
  constructor(IOmnibridge _bridge,
    IWETH _weth,
    address _owner) WETHOmnibridgeRouter(_bridge, _weth, _owner) {}
  function register(Account memory _account) public {
    require(_account.owner == msg.sender, "only owner can be registered");
    _register(_account);
  }
  function wrapAndRelayTokens(address _receiver,
    bytes memory _data,
    Account memory _account) public payable {
    WETH.deposit{ value: msg.value }();
    bridge.relayTokensAndCall(address(WETH), _receiver, msg.value, _data);
    if (_acco

from datasets import load_dataset
dataset = load_dataset("braindao/solidity-dataset")
dataset

In [ ]:














import pandas as pd

# Convert the dataset to a DataFrame
train_df = pd.DataFrame(dataset["train"])

# Display the first few rows
print(train_df.head())

# Display the entire dataset (use with caution for large datasets)
print(train_df)


                                              expert  \
0  Create a smart contract that extends the WETHO...   
1  Implement a smart contract that inherits from ...   
2  Implement a smart contract that inherits from ...   
3  Create a Solidity smart contract that allows f...   
4  Design a smart contract that implements a crow...   

                                             average  \
0  Create a smart contract that builds upon the W...   
1  Create a smart contract for a BlindAuctionMark...   
2  Create a smart contract that allows for basic ...   
3  Create a smart contract for managing decentral...   
4  Create a smart contract that allows for a cryp...   

                                            beginner  \
0  Create a smart contract that allows users to r...   
1  Design a smart contract for a BlindAuctionMark...   
2  Create a smart contract that enables users to ...   
3  Create a smart contract that allows for the de...   
4  Design a smart contract for a cryptocurrenc

In [ ]:
import pandas as pd

# Example: Assuming the dataset is a DataFrame
# Load your dataset into a DataFrame
data = pd.DataFrame({
    "expert": [
        "Create a smart contract that extends the WETHOptimized contract.",
        "Implement a smart contract that inherits from ERC721.",
        "Implement a smart contract that inherits from ERC721 with metadata.",
        "Create a Solidity smart contract for token staking.",
        "Design a smart contract for a crowdfunding platform."
    ],
    "content": [
        "// contracts/bridge/L1Helper.sol\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract L1Helper { ... }",
        "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract ERC721Metadata is ERC721 { ... }",
        "// SPDX-License-Identifier: UNLICENSED\npragma solidity ^0.8.0; contract ERC721Advanced { ... }",
        "pragma solidity 0.4.24;\nimport '../kernel/IKernel.sol'; contract TokenStaking { ... }",
        "pragma solidity ^0.4.18;\nlibrary SafeMath {...} contract Crowdfunding { ... }"
    ]
})

# Extract only the 'content' column
content_list = data["content"].tolist()

# Print the extracted content
for i, content in enumerate(content_list[:5]):  # Display only the first 5 for brevity
    print(f"Smart Contract {i + 1}:\n{content}\n")


Smart Contract 1:
// contracts/bridge/L1Helper.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0; contract L1Helper { ... }

Smart Contract 2:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0; contract ERC721Metadata is ERC721 { ... }

Smart Contract 3:
// SPDX-License-Identifier: UNLICENSED
pragma solidity ^0.8.0; contract ERC721Advanced { ... }

Smart Contract 4:
pragma solidity 0.4.24;
import '../kernel/IKernel.sol'; contract TokenStaking { ... }

Smart Contract 5:
pragma solidity ^0.4.18;
library SafeMath {...} contract Crowdfunding { ... }



In [ ]:
import os

# Create a directory to store the Solidity contracts
os.makedirs("solidity_contracts", exist_ok=True)

# Save each contract to a separate file
for i, content in enumerate(content_list):
    with open(f"solidity_contracts/contract_{i + 1}.sol", "w") as file:
        file.write(content)

print(f"Saved {len(content_list)} contracts to the 'solidity_contracts' directory.")


Saved 5 contracts to the 'solidity_contracts' directory.


In [ ]:
import os

# Create a directory to store the Solidity contracts
os.makedirs("solidity_contracts", exist_ok=True)

# Save each contract to a separate file
for i, content in enumerate(content_list):
    with open(f"solidity_contracts/contract_{i + 1}.sol", "w") as file:
        file.write(content)

print(f"Saved {len(content_list)} contracts to the 'solidity_contracts' directory.")


Saved 5 contracts to the 'solidity_contracts' directory.


from datasets import Dataset


data = {
    "expert": [
        "Create a smart contract that extends the WETHOptimized contract and adds a transfer function.",
        "Implement a smart contract that inherits from ERC721 and adds token metadata functionality.",
        "Implement a smart contract that inherits from ERC721 and adds token metadata functionality.",
        "Create a Solidity smart contract that allows for token staking.",
        "Design a smart contract that implements a crowdfunding platform.",
        "Create a smart contract that implements the BKC token.",
        "Implement a smart contract that simulates a Coin Flip game."
    ],
    "content": [
        "// contracts/bridge/L1Helper.sol\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract L1Helper { ... }",
        "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract ERC721Metadata is ERC721 { ... }",
        "// SPDX-License-Identifier: UNLICENSED\npragma solidity ^0.8.0; contract ERC721Advanced { ... }",
        "pragma solidity 0.4.24;\nimport '../kernel/IKernel.sol'; contract TokenStaking { ... }",
        "pragma solidity ^0.4.18;\nlibrary SafeMath {...} contract Crowdfunding { ... }",
        "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0; contract BKCToken { ... }",
        "// SPDX-License-Identifier: AGPL-3.0-only\npragma solidity ^0.8.0; contract CoinFlip { ... }"
    ]
}


dataset = Dataset.from_dict(data)

content_list = [record["content"] for record in dataset]

for i, content in enumerate(content_list):
    print(f"Smart Contract {i + 1}:\n{content}\n")

from datasets import load_dataset
import os

# Load the braindao/solidity-dataset from Hugging Face
dataset = load_dataset("braindao/Solidity-Dataset")

# Access the training split
train_dataset = dataset["train"]

# Extract the Solidity code from the 'content' field
solidity_code_list = train_dataset["content"]

# Display the first few contracts
for i, code in enumerate(solidity_code_list[:5]):  # Adjust the range as needed
    print(f"Smart Contract {i + 1}:\n{code}\n")

# Save Solidity contracts to individual .sol files
output_dir = "solidity_contracts"
os.makedirs(output_dir, exist_ok=True)

for i, code in enumerate(solidity_code_list):
    file_path = os.path.join(output_dir, f"contract_{i + 1}.sol")
    with open(file_path, "w") as file:
        file.write(code)

print(f"Extracted {len(solidity_code_list)} Solidity contracts and saved to '{output_dir}' directory.")


In [ ]:
from datasets import load_dataset

# Load the braindao/solidity-dataset
dataset = load_dataset("braindao/Solidity-Dataset")

# Access the training split
train_dataset = dataset["train"]

# Extract the 'content' field containing Solidity code
solidity_code_list = train_dataset["content"]

# Format and display all contracts
output = []
for i, code in enumerate(solidity_code_list, start=1):
    formatted_contract = f"Smart Contract {i}:\n{code}\n"
    output.append(formatted_contract)
    print(formatted_contract)

# Save to a file
with open("formatted_smart_contracts.txt", "w") as file:
    file.writelines(output)

print("Formatted smart contracts saved to 'formatted_smart_contracts.txt'.")
